In [2]:
import sys
import time
import jsondiff
import pandas as pd
import os
import re
from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json, jsondiff
from sklearn.model_selection import train_test_split
import copy
from dataclasses import dataclass, asdict
from typing import Any, Dict, List, Tuple, Optional

In [3]:
my_models_real=[
    "codegemma:7b",
    "codestral:22b",
    "codellama:34b",
    "codellama:7b",
    "command-r:35b",
    "deepseek-coder:1.3b",
    "Deepseek-coder-v2:16b",
    "dolphin-mistral:7b",
    "gemma2:27b",
    "huihui_ai/qwq-abliterated:latest",
    "huihui_ai/qwq-fusion:latest",
    "llama2:7b",
    "Llama3:8b",
    "llama3.1:8b",
    "llama3.2:3b",
    "llava-llama3:8b",
    "marco-o1:7b",
    "mistral:latest",
    "mistral-nemo:12b",
    "openchat:7b",
    "orca-mini:3b",
    "phi:2.7b",
    "phi3:3.8b",
    "qwen:4b",
    "qwen2:7b",
    "qwen2.5:7b",
    "qwq:latest",
    "starcoder:3b",
    "starcoder2:3b",
    "TinyLlama:1.1b",
    "wizardlm2:7b",
    "yi:6b",
    "zephyr:7b"
]

very_large_models = [
"llama3.3:latest",
"llama2:70b",
"codellama:70b"]

default_model = "llama2:7b"

num_context_examples = [0,1,3,6,9]

ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

In [4]:
TRANSLATION_PROMPT_FLOODLIGHT = """
You are a meticulous network engineer. Convert the user's intent into a single Floodlight (Static Entry Pusher, OpenFlow 1.0) flow rule as a JSON object ONLY (no code fences, no comments, no extra text).

Return exactly ONE JSON object with ONLY these keys.

Required keys (all strings):
- "switch": "00:00:00:00:00:00:00:0X"   (DPID; e.g., N → "...:0N")
- "name": "flow-<short-descriptor>"
- "active": "true"
- "priority": "<0-65535>"

Allowed match keys (strings; include ONLY those needed):
- "in_port"
- "eth_type"            (IPv4=0x0800, IPv6=0x86dd, ARP=0x0806)
- "eth_src", "eth_dst"
- "ip_proto"            (TCP=6, UDP=17, ICMP=1)
- "ipv4_src", "ipv4_dst", "ipv6_src", "ipv6_dst"
- "tcp_src", "tcp_dst", "udp_src", "udp_dst", "sctp_src", "sctp_dst"
- "icmpv4_type", "icmpv4_code"
- "arp_opcode", "arp_spa", "arp_tpa", "arp_sha", "arp_tha"
- "eth_vlan_vid", "eth_vlan_pcp"

Allowed action key:
- "actions": "<comma-separated actions>" (omit this key entirely to DROP)

Allowed action values (comma-separated in a single string):
- "output=X"
- "set_queue=Y"
- "push_vlan=0x8100"
- "pop_vlan"
- "set_vlan_vid=VID"

Interpretation rules:
- “in/on switch N” → "switch":"00:00:00:00:00:00:00:0N".
- If you use any IP/L4 fields and "eth_type" is unspecified, set "eth_type":"0x0800".
- Protocol cues: TCP→"ip_proto":"6", UDP→"17", ICMP→"1".
- Services: HTTP→"tcp_dst":"80", HTTPS→"443", SSH→"22", DNS→"udp_dst":"53", DHCP→"udp_dst":"67" (server) or "68" (client), ping→"icmpv4_type":"8".
- “forward/send out/through port X” → "actions":"output=X".
- QoS “queue Y” → "actions":"set_queue=Y,output=X".
- “block/deny/drop” → OMIT the "actions" key.
- Use ONLY fields implied by the intent (plus the defaults above). All keys and values are strings.

Priority guidance (pick the most appropriate):
- "300" → explicit block/deny
- "200" → highly specific forwarding with QoS
- "100" → specific L3/L4 matches
- "50"  → less-specific catch-alls

Constraints:
- Output must be a single valid JSON object, nothing else.
- Do not invent fields, do not include comments or markdown.
"""


In [5]:
# === FLOODLIGHT CANONICALIZATION + COMPARATORS ===

# Normalizations
PROTO_MAP = {"tcp":6, "udp":17, "icmp":1, "icmpv4":1, "icmpv6":58}
ETH_TYPE_ALIASES = {"0x800":2048,"0x0800":2048,2048:2048,"2048":2048,"0x86dd":34525,"34525":34525,34525:34525}
MATCH_KEY_SYNONYMS = {"icmp_type":"icmpv4_type","nw_src":"ipv4_src","nw_dst":"ipv4_dst","dl_type":"eth_type"}

RESERVED_KEYS = {"switch", "name", "active", "priority", "actions"}

def _to_int_maybe(x: Any) -> Any:
    if isinstance(x, bool): return x
    if isinstance(x, (int, float)): return int(x) if isinstance(x,float) and x.is_integer() else x
    if isinstance(x, str):
        s = x.strip()
        if s.lower().startswith("0x"):
            try: return int(s,16)
            except: return x
        if re.fullmatch(r"-?\\d+", s):
            try: return int(s)
            except: return x
        if s.lower() in PROTO_MAP: return PROTO_MAP[s.lower()]
        if s.lower() in ETH_TYPE_ALIASES: return ETH_TYPE_ALIASES[s.lower()]
        return s
    return x

def canonicalize_floodlight_actions(actions_str: Optional[str]) -> List[Dict[str, Any]]:
    if actions_str is None:
        return [] # This is the VALID "drop" action (key was omitted)
    
    if actions_str == "":
        # This is an explicit empty string, which is an error.
        # We represent it as a "bad" action so it fails the comparison.
        return [{"type": "INVALID_EMPTY_ACTION_STRING"}]
    
    parts = str(actions_str).split(',')
    norm = []
    for part in parts:
        if not part: continue
        if "=" in part:
            key, val = part.split('=', 1)
            key = key.strip().upper()
            val = _to_int_maybe(val.strip())
            if key == "OUTPUT":
                norm.append({"type": "OUTPUT", "port": val})
            elif key == "SET_QUEUE":
                norm.append({"type": "SET_QUEUE", "queue_id": val})
            elif key == "SET_VLAN_VID":
                norm.append({"type": "SET_FIELD", "field": "vlan_vid", "value": val})
            elif key == "PUSH_VLAN":
                 # Handle "push_vlan=0x8100"
                norm.append({"type": "PUSH_VLAN", "ethertype": val})
            else:
                norm.append({"type": key, "value": val})
        else:
             # Handle "pop_vlan"
            norm.append({"type": part.strip().upper()})

    # de-dup + deterministic order
    def sig(d): return tuple(sorted(d.items()))
    seen, uniq = set(), []
    for a in norm:
        s = sig(a)
        if s not in seen:
            seen.add(s); uniq.append(a)
    uniq.sort(key=lambda d: (d.get("type",""), json.dumps(d, sort_keys=True)))
    return uniq

def canonicalize_floodlight_rule(obj: Dict[str, Any]) -> Dict[str, Any]:
    if not isinstance(obj, dict): return {}

    # Canonicalize Switch DPID
    dpid = None
    sw_str = obj.get("switch")
    if sw_str:
        try: dpid = int(sw_str.replace(":", ""), 16)
        except: dpid = _to_int_maybe(sw_str)

    # Canonicalize Priority
    priority = _to_int_maybe(obj.get("priority"))

    # Canonicalize Match Fields (all keys that are NOT reserved)
    match = {}
    for k, v in obj.items():
        if k in RESERVED_KEYS:
            continue
        key = MATCH_KEY_SYNONYMS.get(k, k)
        match[key] = _to_int_maybe(v)

    # Canonicalize Actions
    actions = canonicalize_floodlight_actions(obj.get("actions"))

    return {
        "dpid": dpid,
        "table_id": 0, # Floodlight API doesn't use table_id in this JSON
        "priority": priority,
        "match": dict(sorted(match.items())),
        "actions": actions
    }

# --- COMPARATOR LOGIC

@dataclass
class CompareConfig:
    require_same_dpid: bool = True
    require_same_table: bool = False
    require_same_priority: bool = False
    allow_pred_match_superset: bool = True
    require_actions_equal: bool = True

def _match_includes(m_pred: Dict[str,Any], m_gt: Dict[str,Any]) -> bool:
    # Ground-truth values must be present in the prediction
    for k,v in m_gt.items():
        if k not in m_pred or m_pred[k] != v: return False
    return True

def _actions_equal(a_pred: List[Dict[str,Any]], a_gt: List[Dict[str,Any]]) -> bool:
    return a_pred == a_gt  # a_pred and a_gt are already canonicalized

def compare_floodlight_json_translation(pred_obj: Dict[str, Any], gt_obj: Dict[str, Any], cfg: Optional[CompareConfig]=None):
    cfg = cfg or CompareConfig()
    # Use the NEW Floodlight canonicalizer
    c_pred, c_gt = canonicalize_floodlight_rule(pred_obj), canonicalize_floodlight_rule(gt_obj)

    details = {
        "dpid_equal": c_pred["dpid"] == c_gt["dpid"],
        "table_equal": c_pred["table_id"] == c_gt["table_id"],
        "priority_equal": c_pred["priority"] == c_gt["priority"],
        "match_includes_gt": _match_includes(c_pred["match"], c_gt["match"]),
        "match_equal": c_pred["match"] == c_gt["match"],
        "actions_equal": _actions_equal(c_pred["actions"], c_gt["actions"]),
        "pred_canonical": c_pred, "gt_canonical": c_gt
    }

    if cfg.require_same_dpid and not details["dpid_equal"]: return False, details
    if cfg.require_same_table and not details["table_equal"]: return False, details
    if cfg.require_same_priority and not details["priority_equal"]: return False, details

    if cfg.allow_pred_match_superset:
        if not details["match_includes_gt"]: return False, details
    else:
        if not details["match_equal"]: return False, details

    if cfg.require_actions_equal and not details["actions_equal"]: return False, details

    return True, details

In [6]:
ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

# Load custom dataset from CSV
custom_dataset = pd.read_csv('Intent2Flow-Floodlight.csv') #dataset for intent translation task.

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

# Split into train and test (50/50 split for example)
trainset, testset = train_test_split(custom_dataset, test_size=0.5, random_state=42, shuffle=True)

# SYSTEM PROMPT (Manually define)
SYSTEM_PROMPT = TRANSLATION_PROMPT_FLOODLIGHT

In [7]:
csv_file = f"Intent_translation_accuracies_Floodlight_{int(time.time())}.csv"
output_file = f"Intent_translation_details_Floodlight_{int(time.time())}.txt"
# Open the file in write mode
file = open(output_file, "w")  # Use "w" for write mode

In [8]:
results = []

for num_examples in num_context_examples:
    for model in my_models_real:

        correct_translations = 0
        total_samples = len(testset)
        processing_times = []

        # create example selector with one example, then clear the data and add all examples
        # this is a trick to reset data and remove data from continuous learning in previous run
        example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
            [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
            ollama_emb,
            Chroma,
            input_keys=["instruction"],
            k=num_examples,
            vectorstore_kwargs={"fetch_k": min(num_examples, len(trainset))}
            )

        # Clear and add all remaining examples from the trainset
        example_selector.vectorstore.reset_collection()
        for _, row in trainset.iterrows():
            example_selector.add_example({
                "instruction": row["instruction"],
                "output": row["output"]
            })

        for _, testcase in testset.iterrows():
            intent = testcase["instruction"]
            expected_output = testcase["output"]
            system_prompt = SYSTEM_PROMPT
            count = 0
            while True:
                count+=1
                try:
                    time.sleep(0.1)
                    current_time = time.time()
                    if num_examples > 0:
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        system_prompt += example_str + "\n\n\n"  
                    
                    response = client.generate(
                        model=model,
                        options={
                            'temperature': 0.6,
                            'num_ctx': 8192,
                            'top_p': 0.3,
                            'num_predict': 1024,
                            'num_gpu': 99,
                            },
                        stream=False,
                        system=system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    actual_output = response['response']
                    proc_time_s = (time.time() - current_time)
                    processing_times.append(proc_time_s)
                    break

                except Exception as e:
                    print(f"Error in generating translation for {model} with {num_examples} examples: {e}") 
                    sys.stdout.flush()
                    
                    if(count<15):
                        continue
                    else:
                        print("\n",model, " failed to produce valid JSON for translation info after 15 tries. Going to next model\n")
                        break

            try:
                expected_output = json.loads(expected_output)
                actual_output = json.loads(actual_output)

                # Check correctness

                cfg = CompareConfig(
                    require_same_dpid=True,
                    require_same_table=False,      # set True if you want strict table compare
                    require_same_priority=False,   # set True if you want strict priority compare
                    allow_pred_match_superset=False,
                    require_actions_equal=True
                )
            
                is_ok, details = compare_floodlight_json_translation(actual_output, expected_output, cfg)

                #if compare_onos_json(expected_output, actual_output):
                if is_ok:
                    correct_translations += 1            

                file.write(f"Input: {intent}")
                file.write("\nExpected Output\n")
                file.write(f"Expected: {expected_output}")
                file.write("\nActual Output\n")
                file.write(f"Actual: {actual_output}")

                # Write the first output
                file.write(f"\nResult: model: {model}, num context examples: {num_examples}, processing time: {round(proc_time_s, 2)}\n")
                # Write the second output
                file.write(f"Diff: {jsondiff.diff(expected_output, actual_output)}\n\n")

            except Exception as e:
                print("Exception found in post-translation processing: ", e)

            print("========End of a testcase===========\n")
        print("\n************Next MODEL***********")
        file.write(f"*************************Next MODEL*************************\n\n")

        # Compute accuracy
        accuracy = round((correct_translations / total_samples) * 100, 2)
        avg_time = round(sum(processing_times) / total_samples, 2)

        # Store results
        results.append({
            "model": model,
            "context_example": num_examples,
            "num_test_samples": total_samples,
            "correct_translations": correct_translations,
            "accuracy (%)": accuracy,
            "avg_time_per_translation (s)": avg_time
            })

# Save to CSV
pd.DataFrame(results).to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False) 


# Close the file explicitly
file.close()

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========

========End of a testcase===========


************Next MODEL***********
========End of 